In [5]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

In [6]:
# Root directory of the project
ROOT_DIR = os.getcwd()
if ROOT_DIR.endswith("tanks"):
    # Go up two levels to the repo root
    ROOT_DIR = os.path.dirname(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
import tanks

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [7]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "dataset_Tank")

# Use configuation from tanks.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(tanks.TanksConfig):
    IMAGE_RESIZE_MODE = "none"
    
config = NoResizeConfig()

AttributeError: module 'tanks' has no attribute 'TanksConfig'

In [21]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Load dataset
dataset = tanks.TanksDataset()
#     train: loads train 
#     val: loads validation images
dataset.load_Tanks(DATASET_DIR, subset="train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))